In [19]:
import kastore
import pandas as pd
import tskit
import numpy as np
import re
from tqdm import tqdm

In [2]:
meta_path = "../../output/sims_info.tsv"
df = pd.read_csv(meta_path, sep="\t", header=None)
sh = pd.read_csv("../../output/header_sims_info.tsv", sep="\t")
cols = sh.columns.to_list()
if cols[-1] != "poscoefdecayeps":
    cols.append("poscoefdecayeps")
df.columns = cols

In [3]:
df = df.astype({"L":int})

In [4]:
parcols = ["delprop", "delcoef", "posprop", "poscoef", "L", "mu", "rescf", "N", "gens"]

In [5]:
rands = df.rand_id.unique()

In [ ]:
for rand in tqdm(rands):
    #if rand == "ZPCJ98FMCC07SSN":
    #    break
    #print(rand)
    for _, row in df[df.rand_id==rand].iterrows():
        #print(row["edge"], rand)
        tspath = f'../../output/{rand}/{row["edge"]}_{rand}_rep0.trees'
        s = kastore.load(tspath)
        prov = tskit.unpack_strings(s.get('provenances/record'), s.get('provenances/record_offset'))
        command = np.array(eval(prov[-1])["parameters"]["command"])
        # only keeping the params
        command = command[["=" in e for e in command]]
        cmd_dict = {}
        for e in command:
            match = re.findall(r'^(\w+)=\"(.+)\"', e)
            assert len(match) <= 1
            #print(match)
            if len(match) == 1:
                cmd_dict[match[0][0]] = match[0][1]
        for par in parcols:
            assert par in cmd_dict
            assert np.isclose(float(cmd_dict[par]), float(row[par]))

 78%|███████▊  | 813/1048 [12:45<04:07,  1.05s/it]